In [9]:
import os
from dotenv import load_dotenv
load_dotenv()
oppenai_api_key = os.getenv("OPENAI_API_KEY")
if not oppenai_api_key:
    print("OPENAI_API_KEY not found in environment variables.")
else:
    print("OPENAI_API_KEY successfully loaded.")

OPENAI_API_KEY successfully loaded.


In [10]:
documents = [
    "The Impact of Artificial Intelligence on Modern Business Operations"
    "Entrepreneurship and Job Creation in the Nigerian Digital Economy"
    "Building a Scalable Full-Stack AI Application Using FastAPI and React"

]

In [11]:
from sentence_transformers import SentenceTransformer
import numpy as np
from openai import OpenAI

model = SentenceTransformer ('all-MiniLM-L6-v2')
doc_embeddings = model.encode(documents)

'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 02144e08-06a3-430c-b322-3e6b559b52ac)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


In [12]:
query = "what is Business?"
query_embedding = model.encode([query])[0]

In [13]:
similarities = np.dot(doc_embeddings, query_embedding)
top_idx = np.argmax(similarities)
retrieved_doc = documents[top_idx]

In [14]:
# 4. Generate response
client = OpenAI(api_key=openai_api_key)
prompt = f"""Context: {retrieved_doc}

Question: {query}

Answer based on the context:"""

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}],
    temperature=1
)

In [15]:
response


ChatCompletion(id='chatcmpl-ClMHnu83GSG92ulh7EDkoMasnHxMI', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Business refers to the activities involved in producing, selling, and trading goods and services in order to generate profit. In the context of the impact of artificial intelligence on modern business operations, entrepreneurship and job creation in the Nigerian digital economy, and building a scalable full-stack AI application using FastAPI and React, business encompasses the use of technology, innovation, and digital tools to drive growth, create new opportunities, and improve efficiency in various industries.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1765403119, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=87, prompt_tokens=56, total_tokens=143, completion_token

In [16]:
print("Custom RAG Answer:")
print(response.choices[0].message.content)

Custom RAG Answer:
Business refers to the activities involved in producing, selling, and trading goods and services in order to generate profit. In the context of the impact of artificial intelligence on modern business operations, entrepreneurship and job creation in the Nigerian digital economy, and building a scalable full-stack AI application using FastAPI and React, business encompasses the use of technology, innovation, and digital tools to drive growth, create new opportunities, and improve efficiency in various industries.


In [ ]:
# !pip install langchain langchain-core langchain-community langchain-openai langchain-text-splitters faiss-cpu python-dotenv

  Using cached langchain-1.1.3-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_core-1.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_openai-1.1.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_text_splitters-1.0.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached faiss_cpu-1.13.1-cp311-cp311-win_amd64.whl.metadata (7.6 kB)
  Using cached langgraph-1.0.4-py3-none-any.whl.metadata (7.8 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached uuid_utils-0.12.0-cp39-abi3-win_amd64.whl.metadata (1.1 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached langgraph_checkpoint-3.0.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached langgraph_prebuilt-1.0.5-py3-none-any.whl.metadata (5.2 kB)
  Using cached langgraph_sdk-0.2.15-py3-none-any.whl.metadata (1.6 kB)
  Using cached xxhash-3.6.0-cp311-cp311-win_amd64.wh

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\user\\Documents\\working with LLM\\benv\\Lib\\site-packages\\langchain_community\\embeddings\\elasticsearch.py'
Check the permissions.




   ------------------------------------- -- 33/35 [langchain-community]
   ------------------------------------- -- 33/35 [langchain-community]
   ------------------------------------- -- 33/35 [langchain-community]

  Using cached langchain-1.1.3-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langgraph-1.0.4-py3-none-any.whl.metadata (7.8 kB)
Using cached langchain-1.1.3-py3-none-any.whl (102 kB)
Using cached langgraph-1.0.4-py3-none-any.whl (157 kB)
Using cached langchain_community-0.4.1-py3-none-any.whl (2.5 MB)

   ---------------------------------------- 0/3 [langgraph]
   ---------------------------------------- 0/3 [langgraph]
   ---------------------------------------- 0/3 [langgraph]
   ---------------------------------------- 0/3 [langgraph]
   ---------------------------------------- 0/3 [langgraph]
   ---------------------------------------- 0/3 [langgraph]
   -----------------------------------

In [21]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Convert documents
lc_docs = [Document(page_content=doc) for doc in documents]

# Vector store
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectorstore = FAISS.from_documents(lc_docs, embeddings)
retriever = vectorstore.as_retriever()

# LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    openai_api_key=openai_api_key
)

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert assistant. Use ONLY the retrieved context."),
    ("human", "{question}\n\nContext:\n{context}")
])

# Build RAG pipeline
rag_chain = (
    RunnableParallel(context=retriever, question=RunnablePassthrough())
    | prompt
    | llm
)

# Query
response = rag_chain.invoke("What is RAG?")
print(response)


content='The context provided does not contain information about "RAG." If you have a specific context or definition in mind, please provide it, and I can assist you further.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 95, 'total_tokens': 129, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'id': 'chatcmpl-ClcOWVpg0JkfGC8zqZkhMPngc57tV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b0dea-8d3b-7683-9013-efa448babf79-0' usage_metadata={'input_tokens': 95, 'output_tokens': 34, 'total_tokens': 129, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [22]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Convert documents
lc_docs = [Document(page_content=doc) for doc in documents]

# Vector store (Chroma)
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# persist_directory allows saving DB locally; optional
vectorstore = Chroma.from_documents(
    lc_docs,
    embeddings,
    collection_name="my_rag_collection",
    persist_directory="./chroma_db"  # optional
)

retriever = vectorstore.as_retriever()

# LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    openai_api_key=openai_api_key
)

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert assistant. Use ONLY the retrieved context."),
    ("human", "{question}\n\nContext:\n{context}")
])

# Build RAG pipeline
rag_chain = (
    RunnableParallel(context=retriever, question=RunnablePassthrough())
    | prompt
    | llm
)

# Query
response = rag_chain.invoke("What is RAG?")
print(response)


content='RAG typically stands for "Retrieval-Augmented Generation," which is a method that combines retrieval of information from a database or knowledge base with generative capabilities of AI models to produce more accurate and contextually relevant responses. However, the provided context does not specifically mention RAG or its definition.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 72, 'total_tokens': 131, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'id': 'chatcmpl-ClcPj7tSRusT0jp1nwlQia0VjxJgV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b0deb-b1d6-7be0-ab77-28db3ad88724-0' usage_metadata={'input_tokens': 

In [23]:
response.content

'RAG typically stands for "Retrieval-Augmented Generation," which is a method that combines retrieval of information from a database or knowledge base with generative capabilities of AI models to produce more accurate and contextually relevant responses. However, the provided context does not specifically mention RAG or its definition.'

In [ ]:
# !pip install llama-index

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   ------------------------

In [26]:
from llama_index.core import Document, VectorStoreIndex, Settings
from llama_index.llms.openai import OpenAI as LlamaOpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Configure LlamaIndex
Settings.llm = LlamaOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=openai_api_key)
Settings.embed_model = OpenAIEmbedding(api_key=openai_api_key)

# Create documents and index
llama_docs = [Document(text=doc) for doc in documents]
index = VectorStoreIndex.from_documents(llama_docs)

# Query
query_engine = index.as_query_engine()
response = query_engine.query("What is RAG?")

print("LlamaIndex Answer:")
print(response.response)

LlamaIndex Answer:
RAG typically stands for Red, Amber, Green, which is a color-coding system used to visually indicate the status of various items or tasks.
